In [1]:
from poisson_fem.mesh import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [2]:
m = 2
mesh = PoissonFEM.RectangularMesh(np.ones(m)/m)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
def essBoundaryFun(x):
    return 1.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

In [4]:
def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

In [5]:
#Define boundary flux field
def flux(x):
    a = np.array([1, 2, 3])
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [6]:
rhs = PoissonFEM.RightHandSide(mesh)

In [7]:
rhs.setNaturalRHS(mesh, flux)

In [8]:
funSpace = PoissonFEM.FunctionSpace(mesh)

In [9]:
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)

In [10]:
rhs.setRhsStencil(mesh, K)

In [11]:
lmbda_np = np.arange(5.0, m**2 + 5.0)
lmbda = PETSc.Vec().createSeq(mesh.nCells)
rangeCells = range(mesh.nCells)
lmbda.setValues(rangeCells, lmbda_np)
rhs.assemble(lmbda)
# diff = rs.naturalRHS - rs.rhs
# print('diff = ', diff.array)
print('rhs = ', rhs.vector.array)

rhs =  [-0.91666667 -0.75       -0.41666667  1.66666667  1.25       -0.25
  1.75        2.        ]


In [12]:
# Precomputations
x = PETSc.Vec().createSeq(mesh.nEq)

In [13]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [14]:
N = 1e0

In [15]:
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly, lmbda = [1, 1, ..., 1]
    K.assemble(lmbda)
    
    # solving
    ksp.setOperators(K.matrix)
    ksp.solve(rhs.vector, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  0.0002651214599609375


In [16]:
# for scipy comparison
Kes = K.globStiffStencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [17]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ lmbda.array
    K1 = sps.csr_matrix((Kvecs[K.vec_nonzero], K.indices, K.indptr))
    x1 = lg.spsolve(K1, rhs.vector.array)
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

scipy time =  0.00089263916015625


In [18]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

result difference =  8.628708758818834e-16
PETSc speedup =  3.366906474820144


In [19]:
print(x.array)

[1.11412454 1.20824632 1.2244766  1.43069943 1.60746186 1.38361102
 1.66285435 1.90792877]


In [20]:
print(x1)

[1.11412454 1.20824632 1.2244766  1.43069943 1.60746186 1.38361102
 1.66285435 1.90792877]
